In [1]:
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
import joblib
import os
import json
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def load_data(train_data_path, test_data_path):
    # Load training and testing data from CSV files
    train_data = pd.read_csv(train_data_path)
    test_data = pd.read_csv(test_data_path)
    return train_data, test_data

def evaluate_model(y_true, y_pred):
    # Calculate RMSE
    rmse = mean_squared_error(y_true, y_pred, squared=False)

    # Calculate R-squared
    r2 = r2_score(y_true, y_pred)

    # Calculate Pearson's Linear Correlation Coefficient
    lcc, _ = pearsonr(y_true, y_pred)

    return rmse, r2, lcc

def leave_one_out_cv(train_data, target_column, max_components):
    errors = []
    features = train_data.drop(columns=[target_column]).columns  # Get feature names

    for n_components in range(1, max_components + 1):
        component_errors = []
        pls = PLSRegression(n_components=n_components)
        for i in range(len(train_data)):
            # Remove the current sample
            X_leave_one_out = train_data.drop(index=i)
            y_leave_one_out = target_column.drop(index=i)

            # Fit PLS model with feature names
            pls.fit(X_leave_one_out, y_leave_one_out)

            # Predict the removed sample
            y_pred = pls.predict([train_data.iloc[i]])[0]

            # Calculate prediction error
            error = mean_squared_error([target_column.iloc[i]], [y_pred])
            component_errors.append(error)

        # Average error for this number of components
        avg_error = np.mean(component_errors)
        errors.append(avg_error)

    return errors


In [5]:
with open('./../data_processing/config.json', 'r') as f:
        config = json.load(f)

    # File paths for training and testing datasets
spectra_type = "mir"  # Adjust as needed


In [6]:
for target_column in config['targets']:
        # Loop over each target column in the configuration file

        # File paths for training and testing datasets for the current target
        train_data_path = f'./../train_test_dataset/train_data_{spectra_type}_{target_column}.csv'
        test_data_path = f'./../train_test_dataset/test_data_{spectra_type}_{target_column}.csv'

        # Load data
        print(f"Loading data for target: {target_column}")
        train_data, test_data = load_data(train_data_path, test_data_path)

        # Perform leave-one-out cross-validation
        print(f"Performing leave-one-out cross-validation...")
        max_components = 200
        errors = leave_one_out_cv(train_data, train_data[target_column], max_components)

        # Plot the results
        plt.plot(range(1, max_components + 1), errors, marker='o')
        plt.xlabel('Number of Latent Variables')
        plt.ylabel('Mean Squared Error (MSE)')
        plt.title(f'Leave-One-Out Cross-Validation for Target: {target_column}')
        plt.show()

        # Train the final PLSR model with the best number of components
        best_n_components = np.argmin(errors) + 1
        print(f"Training the final PLSR model with {best_n_components} components...")
        final_model = PLSRegression(n_components=best_n_components)
        final_model.fit(train_data.drop(columns=[target_column]), train_data[target_column])

        # Save the trained model
        output_model_path = f'./../trained_models/plsr_model_{spectra_type}_{target_column}.joblib'
        os.makedirs(os.path.dirname(output_model_path), exist_ok=True)
        joblib.dump(final_model, output_model_path)
        print(f"Trained PLSR model saved to: {output_model_path}")


Loading data for target: c.tot_usda.a622_w.pct
Performing leave-one-out cross-validation...


KeyError: '[(1.73451, 24.37284, 3.37292, 1.32085, 2.65666, 21.232, 1.49225, 1.421, 20.16761, 2.21113, 1.336, 2.26142, 2.12991, 4.3721, 11.55455, 0.35512, 5.77039, 0.3111, 1.20641, 1.1596, 0.82563, 1.5745, 4.65076, 0.7172, 2.12502, 4.75298, 0.90304, 0.25627, 0.660631955, 3.1402, 7.58786, 0.33131, 1.07231, 10.70464, 6.57543, 0.79577, 25.85896, 0.5655, 3.89043, 0.65463, 2.45203930478, 0.62973, 16.25114, 0.36665, 0.27065, 2.51071, 0.33072, 3.07132, 1.48, 4.63344, 0.71762, 0.618357360363007, 1.47212, 1.94155, 2.34439, 0.65425, 0.54876, 2.62866, 0.54303, 5.772, 1.82577, 0.90996, 0.8124, 1.20924, 5.912, 17.71346, 0.83267, 1.29477, 13.87861, 1.26125, 0.47578, 4.10818, 1.59053, 0.70425, 0.6152, 0.3674, 2.2534, 0.59169, 0.27048, 0.26668, 0.53984, 1.00432, 3.32238, 24.83015, 3.9463, 0.52803, 0.57168, 2.44017, 2.41438, 2.54344, 28.23637, 1.41528, 0.49123, 0.59821, 1.98082, 1.80051, 0.33889, 0.541, 13.19013, 0.88128, 3.58999, 0.34999, 2.36425, 15.43119, 1.05348, 3.22829, 0.583, 0.81226, 1.37501, 2.